In [1]:
import io
import contextlib 
from ChatGLM3_router import ChatGLM3
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [2]:
# 主要创建三个知识域的模板，一个营销文案作者，一个客服，一个小红书

Copywriter_template = """You are a master of persuasion and a pro at captivating audiences. \
You have the ability to craft captivating messages that move people to action. \
You know that the key to effective communication is understanding your audience and speaking directly to their needs and desires.\

Here is a question:
{input}"""

customerService_template = """You are a customer service representative. You are skilled at assisting customers and addressing their concerns. \
You take a problem-solving approach to help resolve issues and provide solutions. You should answer product, brand questions and purchase questions，and introduce product features.

Here is a question:
{input}"""


In [3]:
prompt_infos = [
    {
        "name": "Copywriter",
        "description": "Good for writing marketing copy ",
        "prompt_template": Copywriter_template,
    },
    {
        "name": "customerService",
        "description": "Good for answering product, brand, purchase and product features questions",
        "prompt_template": customerService_template,
    },
]
model_path = "/data/tds/ChatGLM3/chatglm3-6b"
llm = ChatGLM3()
llm.load_model(model_name_or_path=model_path)

# 下面这段主要通过遍历得到destination_chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
# 得到一个起始chain
default_chain = ConversationChain(llm=llm, output_key="text")


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [94]:
from langchain.chains.router.embedding_router import EmbeddingRouterChain
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Chroma
#import os
#os.environ["COHERE_API_KEY"]="YSXlFqdn6rykzQZ4GYReEBCA8DKudVcytF0tr5M1"
names_and_descriptions = [
    ("Copywriter", ["文案脚本标题软文广告写作"]),
    ("customerService", ["知识问答任务"]),
]
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
embed_model = LangchainEmbedding(embeddings)
#router_chain = EmbeddingRouterChain.from_names_and_descriptions(
#    names_and_descriptions, Chroma, CohereEmbeddings(), routing_keys=["input"]
#)
router_chain = EmbeddingRouterChain.from_names_and_descriptions(
    names_and_descriptions, Chroma, embed_model, routing_keys=["input"]
)

In [95]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [99]:
import pandas as pd
df = pd.read_excel("/home/kit/kit/ChatGLM3/langchain_demo/NewPromptRouter.xlsx",header=None)
df = df.assign(direct0 = None,direct1 = None,direct2 = None,direct3 = None,direct4 = None,direct5 = None)
df

,0,direct0,direct1,direct2,direct3,direct4,direct5
0,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None,None,None,None,None
1,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None,None,None,None,None
2,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None,None,None,None,None
3,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,None,None,None,None,None,None
4,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： LOHO新款眼睛\n\...,None,None,None,None,None,None
...,...,...,...,...,...,...,...
83,Dr.Arrivo美容仪的宙斯二代有哪些特点？,None,None,None,None,None,None
84,TriPollar美容仪对修复肌肤、美白、抑痘的效果如何？,None,None,None,None,None,None
85,JOVS美容仪在家用深层激活方面表现如何？,None,None,None,None,None,None
86,金稻（K·SKIN）美容仪的便携轻巧特点对去黄提亮有何帮助？,None,None,None,None,None,None


In [98]:
for i in range(len(df)):
    print(
            chain.run(
                df.iloc[i,0]
            )
        )



> Entering new MultiPromptChain chain...
Copywriter: {'input': '你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品牌洗面奶 \n\n目标用户： 20-35岁的都市女性 \n\n内容要求： 自然融入产品参数和功能解读，结合用户故事或产品体验 \n\n创作要点： 隐蔽推广，文章主题与产品关联不明显 '}You are a master of persuasion and a pro at captivating audiences. You have the ability to craft captivating messages that move people to action. You know that the key to effective communication is understanding your audience and speaking directly to their needs and desires.
Here is a question:
你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：

产品名称： X品牌洗面奶 

目标用户： 20-35岁的都市女性 

内容要求： 自然融入产品参数和功能解读，结合用户故事或产品体验 

创作要点： 隐蔽推广，文章主题与产品关联不明显 

> Finished chain.



> Entering new MultiPromptChain chain...
Copywriter: {'input': '你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品牌沐浴露 \n\n目标用户： 20-35岁的都市女性 \n\n内容要求： 自然融入产品参数和功能解读，结合用户故事或产品体验 \n\n创作要点： 隐蔽推广，文章主题与产品关联不明显'}You are a master of persuasion and a pro at captivating audiences. You have the ability to craft captivating messages that move people to action.

In [100]:
#输入user_question,判断句子路由方向，返回1代表生成文案，返回0代表知识问答
def get_direct(user_question,chain):
    with contextlib.redirect_stdout(io.StringIO()) as f:
        print(
            chain.run(
                user_question
            )
        )
        ans = f.getvalue() 
    sign = ans[51:61]
    if sign == "Copywriter" or sign == "Red_Writer":
        return 1
    else:
        return 0
get_direct(df.iloc[4,0],chain)

1

In [102]:
for j in range(6):
    for i in range(int(len(df))):
        direct = get_direct(df.iloc[i,0],chain)
        df.iloc[i,j+1] = direct

In [103]:
df.to_excel("Router_output.xlsx")
df

,0,direct0,direct1,direct2,direct3,direct4,direct5
0,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,1,1,1,1,1
1,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,1,1,1,1,1
2,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,1,1,1,1,1
3,你是一位出色的软文写手，请根据以下要求，为一个产品撰写软性广告文章：\n\n产品名称： X品...,1,1,1,1,1,1
4,你是一名广告文案创作者，请根据以下要求创作宣传语：\n\n产品名称： LOHO新款眼睛\n\...,1,1,1,1,1,1
...,...,...,...,...,...,...,...
83,Dr.Arrivo美容仪的宙斯二代有哪些特点？,0,0,0,0,0,0
84,TriPollar美容仪对修复肌肤、美白、抑痘的效果如何？,0,0,0,0,0,0
85,JOVS美容仪在家用深层激活方面表现如何？,0,0,0,0,0,0
86,金稻（K·SKIN）美容仪的便携轻巧特点对去黄提亮有何帮助？,0,0,0,0,0,0
